# Getting started

In this notebook, we illustrate how to use the Neural News Recommendation with Multi-Head Self-Attention ([NRMS](https://aclanthology.org/D19-1671/)). The implementation is taken from the [recommenders](https://github.com/recommenders-team/recommenders) repository. We have simply stripped the model to keep it cleaner.

We use a small dataset, which is downloaded from [recsys.eb.dk](https://recsys.eb.dk/). All the datasets are stored in the folder path ```~/ebnerd_data/*```.

## Load functionality

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataclasses import dataclass, field
from pathlib import Path

import numpy as np
import polars as pl
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel

from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from ebrec.models.newsrec.model_config import hparams_nrms, hparams_naml
from ebrec.models.newsrec.naml import NAMLModel
from ebrec.utils._articles import (
    create_article_id_to_value_mapping,
    convert_text2encoding_with_transformers
)
from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history,
    create_user_id_to_int_mapping
)
from ebrec.utils._constants import (
    DEFAULT_ARTICLE_ID_COL,
    DEFAULT_CATEGORY_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
)
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._python import create_lookup_dict, time_it, write_submission_file, rank_predictions_by_score, create_lookup_objects
from ebrec.utils._articles_behaviors import map_list_article_id_to_value

# Initiate the dataloaders
In the implementations we have disconnected the models and data. Hence, you should built a dataloader that fits your needs.

In [3]:
@dataclass
class NewsrecDataLoader(tf.keras.utils.Sequence):
    """
    A DataLoader for news recommendation.
    """

    behaviors: pl.DataFrame
    history_column: str
    article_dict: dict[int, any]
    unknown_representation: str
    eval_mode: bool = False
    batch_size: int = 32
    inview_col: str = DEFAULT_INVIEW_ARTICLES_COL
    labels_col: str = DEFAULT_LABELS_COL
    user_col: str = DEFAULT_USER_COL
    kwargs: field(default_factory=dict) = None

    def __post_init__(self):
        """
        Post-initialization method. Loads the data and sets additional attributes.
        """
        self.lookup_article_index, self.lookup_article_matrix = create_lookup_objects(
            self.article_dict, unknown_representation=self.unknown_representation
        )
        self.unknown_index = [0]
        self.X, self.y = self.load_data()
        if self.kwargs is not None:
            self.set_kwargs(self.kwargs)

    def __len__(self) -> int:
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self):
        raise ValueError("Function '__getitem__' needs to be implemented.")

    def load_data(self) -> tuple[pl.DataFrame, pl.DataFrame]:
        X = self.behaviors.drop(self.labels_col).with_columns(
            pl.col(self.inview_col).list.len().alias("n_samples")
        )
        y = self.behaviors[self.labels_col]
        return X, y

    def set_kwargs(self, kwargs: dict):
        for key, value in kwargs.items():
            setattr(self, key, value)


@dataclass(kw_only=True)
class NAMLDataLoader(NewsrecDataLoader):
    """
    Eval mode not implemented
    """

    unknown_category_value: int = 0
    unknown_subcategory_value: int = 0
    body_mapping: dict[int, list[int]] = None
    category_mapping: dict[int, int] = None
    subcategory_mapping: dict[int, int] = None

    def __post_init__(self):
        self.title_prefix = "title_"
        self.body_prefix = "body_"
        self.category_prefix = "category_"
        self.subcategory_prefix = "subcategory_"
        (
            self.lookup_article_index_body,
            self.lookup_article_matrix_body,
        ) = create_lookup_objects(
            self.body_mapping, unknown_representation=self.unknown_representation
        )
        # if self.eval_mode:
        #     raise ValueError("'eval_mode = True' is not implemented for NAML")

        return super().__post_init__()

    def transform(self, df: pl.DataFrame) -> tuple[pl.DataFrame]:
        """
        Special case for NAML as it requires body-encoding, verticals, & subvertivals
        """
        # =>
        title = df.pipe(
            map_list_article_id_to_value,
            behaviors_column=self.history_column,
            mapping=self.lookup_article_index,
            fill_nulls=self.unknown_index,
            drop_nulls=False,
        ).pipe(
            map_list_article_id_to_value,
            behaviors_column=self.inview_col,
            mapping=self.lookup_article_index,
            fill_nulls=self.unknown_index,
            drop_nulls=False,
        )
        # =>
        body = df.pipe(
            map_list_article_id_to_value,
            behaviors_column=self.history_column,
            mapping=self.lookup_article_index_body,
            fill_nulls=self.unknown_index,
            drop_nulls=False,
        ).pipe(
            map_list_article_id_to_value,
            behaviors_column=self.inview_col,
            mapping=self.lookup_article_index_body,
            fill_nulls=self.unknown_index,
            drop_nulls=False,
        )
        # =>
        category = df.pipe(
            map_list_article_id_to_value,
            behaviors_column=self.history_column,
            mapping=self.category_mapping,
            fill_nulls=self.unknown_category_value,
            drop_nulls=False,
        ).pipe(
            map_list_article_id_to_value,
            behaviors_column=self.inview_col,
            mapping=self.category_mapping,
            fill_nulls=self.unknown_category_value,
            drop_nulls=False,
        )
        # =>
        subcategory = df.pipe(
            map_list_article_id_to_value,
            behaviors_column=self.history_column,
            mapping=self.subcategory_mapping,
            fill_nulls=self.unknown_subcategory_value,
            drop_nulls=False,
        ).pipe(
            map_list_article_id_to_value,
            behaviors_column=self.inview_col,
            mapping=self.subcategory_mapping,
            fill_nulls=self.unknown_subcategory_value,
            drop_nulls=False,
        )
        return (
            pl.DataFrame()
            .with_columns(title.select(pl.all().name.prefix(self.title_prefix)))
            .with_columns(body.select(pl.all().name.prefix(self.body_prefix)))
            .with_columns(category.select(pl.all().name.prefix(self.category_prefix)))
            .with_columns(
                subcategory.select(pl.all().name.prefix(self.subcategory_prefix))
            )
        )
    
    def __getitem__(self, idx) -> tuple[tuple[np.ndarray], np.ndarray]:
        batch_X = self.X[idx * self.batch_size : (idx + 1) * self.batch_size].pipe(
            self.transform
        )
        
        batch_y = self.y[idx * self.batch_size : (idx + 1) * self.batch_size]
        # =>
        
        batch_y = np.array(batch_y.to_list())
        his_input_title = np.array(batch_X[self.title_prefix + self.history_column].to_list())
        his_input_body = np.array(batch_X[self.body_prefix + self.history_column].to_list())

        # =>
        pred_input_title = np.array(batch_X[self.title_prefix + self.inview_col].to_list())   
        pred_input_body = np.array(batch_X[self.body_prefix + self.inview_col].to_list())

        # =>
        his_input_title = np.squeeze(self.lookup_article_matrix[his_input_title], axis=2)
        pred_input_title = np.squeeze(self.lookup_article_matrix[pred_input_title], axis=2)
        his_input_body = np.squeeze(self.lookup_article_matrix_body[his_input_body], axis=2)
        pred_input_body = np.squeeze(self.lookup_article_matrix_body[pred_input_body], axis=2)
        # =>

        his_input_vert = np.array(batch_X[self.category_prefix + self.history_column].to_list())[:, :, np.newaxis]
        his_input_subvert = np.array(batch_X[self.subcategory_prefix + self.history_column].to_list())[:, :, np.newaxis]
        pred_input_vert = np.array(batch_X[self.category_prefix + self.inview_col].to_list())[:, :, np.newaxis]
        pred_input_subvert = np.array(batch_X[self.subcategory_prefix + self.inview_col].to_list())[:, :, np.newaxis]

        if self.eval_mode: # Added the eval_mode condition
            return (
                his_input_title,
                his_input_body,
                his_input_vert,
                his_input_subvert,
                pred_input_title,
                pred_input_body,
                pred_input_vert,
                pred_input_subvert,
            ), 
        else:
            return (
                his_input_title,
                his_input_body,
                his_input_vert,
                his_input_subvert,
                pred_input_title,
                pred_input_body,
                pred_input_vert,
                pred_input_subvert,
            ), batch_y

In [18]:
import pandas as pd
import math
from collections import Counter  
def make_data_loader(PATH_DATA, do_eval):
        
    TOKEN_COL = "tokens"
    N_SAMPLES = "n"
    BATCH_SIZE = 100
    df_articles = (
        pl.scan_parquet(PATH_DATA.joinpath("../articles.parquet"))
        .select(pl.col(DEFAULT_ARTICLE_ID_COL, DEFAULT_CATEGORY_COL))
        .with_columns(pl.Series(TOKEN_COL, np.random.randint(0, 20, (1, 10))))
        .collect()
    )
    df_history = (
        pl.scan_parquet(PATH_DATA.joinpath("history.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL)
        .with_columns(pl.col(DEFAULT_HISTORY_ARTICLE_ID_COL).list.tail(3))
    )
    df_behaviors = (
        pl.scan_parquet(PATH_DATA.joinpath("behaviors.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_INVIEW_ARTICLES_COL, DEFAULT_CLICKED_ARTICLES_COL)
        .with_columns(pl.col(DEFAULT_INVIEW_ARTICLES_COL).list.len().alias(N_SAMPLES))
        .join(df_history, on=DEFAULT_USER_COL, how="left")
        .collect()
        .pipe(create_binary_labels_column)
    )
    # => MAPPINGS:
    article_mapping = create_article_id_to_value_mapping(
        df=df_articles, value_col=TOKEN_COL
    )
    user_mapping = create_user_id_to_int_mapping(df=df_behaviors)
    # => NPRATIO IMPRESSION - SAME LENGTHS:
    #df_behaviors_train = df_behaviors.filter(pl.col(N_SAMPLES) == pl.col(N_SAMPLES).min())

    min_length = 10000
    for i in range(len(df_behaviors["article_ids_inview"])):
        if len(df_behaviors["article_ids_inview"][i]) < min_length:
            min_length = len(df_behaviors["article_ids_inview"][i])

    article_ids_inview = []
    labels = []
    user_id = []
    article_ids_clicked = []
    article_id_fixed = []
    n = []
    count = 0
    for i in range(len(df_behaviors["article_ids_inview"])):
        full_article_ids_inview = df_behaviors["article_ids_inview"][i].to_list()
        full_labels = df_behaviors["labels"][i].to_list()
        full_article_ids_clicked = df_behaviors["article_ids_clicked"][i].to_list()
        full_article_id_fixed = df_behaviors["article_id_fixed"][i].to_list()
        
        if not full_article_ids_inview or not full_labels or not full_article_ids_clicked or not full_article_id_fixed:
          continue  # Skip the rest of the loop for this iteration

        if len(full_article_ids_inview) == min_length:
            article_ids_inview.append(full_article_ids_inview)
            labels.append(full_labels)
            article_ids_clicked.append(full_article_ids_clicked)
            article_id_fixed.append(full_article_id_fixed)
            user_id.append(df_behaviors["user_id"][i])
            n.append(df_behaviors["n"][i])
        else:
            try:
                for id in full_article_ids_clicked:
                    if id in full_article_ids_inview:
                        full_article_ids_inview.remove(id)
            except ValueError as e:
                continue
            if round((len(full_article_ids_clicked)/len(full_article_ids_inview)) * min_length) >= 1:
                count += 1
            amount_to_remove = round((len(full_article_ids_clicked)/len(full_article_ids_inview)) * min_length)
            article_ids_inview.append(full_article_ids_inview[:(min_length-amount_to_remove)] + full_article_ids_clicked[:amount_to_remove])
            labels.append([0]*(min_length-amount_to_remove) + [1] * amount_to_remove)
            article_ids_clicked.append(full_article_ids_clicked[:amount_to_remove])
            article_id_fixed.append(full_article_id_fixed)
            user_id.append(df_behaviors["user_id"][i])
            n.append(df_behaviors["n"][i])

            # Generate a permutation index (Shuffle the data)
            if len(labels[-1]) != len(article_ids_inview[-1]):
                #print("Length of labels and article_ids_inview is not the same")
                continue
            indices = np.arange(len(labels[-1]))
            np.random.shuffle(indices)
            article_ids_inview[-1] = [article_ids_inview[-1][i] for i in indices]
            labels[-1] = [labels[-1][i] for i in indices]
        
        if len(article_ids_inview[-1]) != min_length:
            article_ids_inview.pop()
            labels.pop()
            article_ids_clicked.pop()
            article_id_fixed.pop()
            user_id.pop()
            n.pop()
        assert len(article_ids_inview[-1]) == min_length, print("Lenght is" , len(article_ids_inview[-1]))

    df_behaviors = pd.DataFrame({"user_id": user_id, "article_ids_inview": article_ids_inview, "article_ids_clicked": article_ids_clicked,
                                 "labels": labels, "n": n, "article_id_fixed":article_id_fixed})
    # Apply padding safely to each list
    df_behaviors_train = pl.from_pandas(df_behaviors)
    print("Length of the dataset", count)
    # Calculate the lengths of each list
    lengths = [len(x) for x in df_behaviors_train['article_ids_clicked']]

    # Count occurrences of each length
    length_counts = Counter(lengths)

    # Print the results
    print("Lengths of 1's in the sequences:")
    for length, count in sorted(length_counts.items()):
        print(f"len {length} = {count}")
    # => MAPPINGS:
    body_mapping = article_mapping
    category_mapping = create_lookup_dict(
        df_articles.select(pl.col(DEFAULT_CATEGORY_COL).unique()).with_row_index(
            "row_nr"
        ),
        key=DEFAULT_CATEGORY_COL,
        value="row_nr",
    )
    subcategory_mapping = category_mapping

    dataloader = NAMLDataLoader(
        behaviors=df_behaviors_train,
        eval_mode=do_eval,
        article_dict=article_mapping,
        body_mapping=body_mapping,
        category_mapping=category_mapping,
        unknown_representation="zeros",
        subcategory_mapping=subcategory_mapping,
        history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
        batch_size=BATCH_SIZE,
    )

    return dataloader, df_behaviors_train

In [19]:
# LOAD DATA:
PATH_DATA = Path("C:/Users/jortv/OneDrive/Bureau/ONCE/ebnerd-benchmark/data/ebnerd_demo/train")
do_eval = False
train_dataloader, df_behaviors_train = make_data_loader(PATH_DATA, do_eval)

Length of the dataset 11255
Lengths of 1's in the sequences:
len 0 = 9111
len 1 = 15549
len 2 = 64


In [20]:
# LOAD DATA:
PATH_DATA = Path("C:/Users/jortv/OneDrive/Bureau/ONCE/ebnerd-benchmark/data/ebnerd_demo/validation")
do_eval = True
val_dataloader, df_behaviors_val = make_data_loader(PATH_DATA, do_eval)

Length of the dataset 11473
Lengths of 1's in the sequences:
len 0 = 9852
len 1 = 15443
len 2 = 60
len 3 = 1


# Train the model


In [43]:
DEFAULT_TITLE_SIZE = 10 #30
DEFAULT_BODY_SIZE = 10 #40
UNKNOWN_TITLE_VALUE = [0] * DEFAULT_TITLE_SIZE
UNKNOWN_BODY_VALUE = [0] * DEFAULT_BODY_SIZE

DEFAULT_DOCUMENT_SIZE = 768


class hparams_naml:
    # INPUT DIMENTIONS:
    title_size: int = DEFAULT_TITLE_SIZE
    history_size: int = 50 #3 #50
    body_size: int = DEFAULT_BODY_SIZE
    vert_num: int = 100
    vert_emb_dim: int = 10
    subvert_num: int = 100
    subvert_emb_dim: int = 10
    # MODEL ARCHITECTURE
    dense_activation: str = "relu"
    cnn_activation: str = "relu"
    attention_hidden_dim: int = 150
    filter_num: int = 400
    window_size: int = 5
    # MODEL OPTIMIZER:
    optimizer: str = "adam"
    loss: str = "cross_entropy_loss"
    dropout: float = 0.2
    learning_rate: float = 0.001


config = hparams_naml()



In [44]:
# Model
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
word2vec_embedding = get_transformers_word_embeddings(transformer_model)

model = NAMLModel(hparams=config, word2vec_embedding=word2vec_embedding)
model.model.fit(train_dataloader, epochs=20, verbose=1)

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/20
248/248 [==============================] - 203s 798ms/step - loss: 1.0195
Epoch 2/20
248/248 [==============================] - 199s 802ms/step - loss: 1.0195
Epoch 3/20
 60/248 [======>.......................] - ETA: 2:31 - loss: 1.0228

KeyboardInterrupt: 

# Example how to compute some metrics:

In [48]:
pred_validation = model.model.predict(val_dataloader)

122/254 [=============>................] - ETA: 9s 

InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.
Traceback (most recent call last):

  File "c:\Users\jortv\anaconda3\envs\baseline_rs\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "c:\Users\jortv\anaconda3\envs\baseline_rs\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\jortv\anaconda3\envs\baseline_rs\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\User

### Compute metrics

In [65]:
df_validation = add_prediction_scores(df_behaviors_val, pred_validation.tolist()).pipe(
    add_known_user_column, known_users=df_behaviors_val[DEFAULT_USER_COL]
)

metrics = MetricEvaluator(
    labels=df_validation["labels"].to_list(),
    predictions=df_validation["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

<MetricEvaluator class>: 
 {
    "auc": 0.4788774973711882,
    "mrr": 0.8613661475806585,
    "ndcg@5": 0.9467854160632154,
    "ndcg@10": 0.9467854160632154
}

: 